In [ ]:
#Put all of your import statements in this code block

# Pipe Table Search Functions

## Search By Inside Diameter

Use this function to get a sorted list of pipe sizes with information regarding
the pipe given a target inside diameter.

In [ ]:
# Always input these variables
inside_diameter = None #Enter the desired inside diameter as a decimal

# Optional variables
standard_only = True #False will include entries without identification or schedule
number_of_entries = None #Enter an integer if you only want to see a certain number of results, default is to see all entries

## Filter by Codes based on pressure and temperature if desired
b31_1 = False #Set to True to only return results that pass B31.1 for design pressure and temperature
b31_3 = False #Set to True to only return results that pass B31.3 for design pressure and temperature
section_vii = False #Set to True to only return results that pass Section VIII Div 1. UG-27 for design pressure and temperature
design_pressure = 0 #psig, must be entered if code filter is set to True
design_temperature = 0 #F, must be entered if code filter is set to True
material = None #'304', '316', '106 Gr B' are the currently available materials, must be entered if code filter is set to True
seamless = True #Set to False to apply a 0.85 strength reduction factor for welded pipe

## Search by Fluid Velocity at Process Conditions
Use this function to get a sorted list of pipe sizes with information regarding the pipe given a target fluid velocity. (Fluid is always assumed to be water currently)

In [ ]:
# Always input these variables
fluid_pressure = None #psia as a decimal, used for density lookup
fluid_temperature = None #F as a decimal , used for density lookup
mass_flow_rate = None #lbm/sec as a decimal
target_velocity = None #ft/sec as a decimal

# Optional variables
standard_only = True #False will include entries without identification or schedule
number_of_entries = None #Enter an integer if you only want to see a certain number of results, default is to see all entries

## Filter by Codes based on pressure and temperature if desired
b31_1 = False #Set to True to only return results that pass B31.1 for design pressure and temperature
b31_3 = False #Set to True to only return results that pass B31.3 for design pressure and temperature
section_vii = False #Set to True to only return results that pass Section VIII Div 1. UG-27 for design pressure and temperature
design_pressure = 0 #psig, must be entered if code filter is set to True
design_temperature = 0 #F, must be entered if code filter is set to True
material = None #'304', '316', '106 Gr B' are the currently available materials, must be entered if code filter is set to True
seamless = True #Set to False to apply a 0.85 strength reduction factor for welded pipe

# Specific Pipe Size Functions

## Information for Specific Pipe Size
Use this function to get information for a particular pipe size. (Fluid is always assumed to be water currently)

In [ ]:
# Always input these variables
nps = None #Enter NPS as a decimal
schedule = None #Only enter schedule or identification. Can be an integer or string
identification = None #Only enter schedule or identification. Can be an integer or string

# Optional variables
fluid_pressure = None #psia as a decimal, used for density lookup
fluid_temperature = None #F as a decimal , used for density lookup
mass_flow_rate = None #lbm/sec as a decimal